# Lab 5: Model Endpoint Testing

This notebook demonstrates how to test the deployed model endpoint from Lab 4.

## Prerequisites
- Complete Lab 4 with approved and deployed model
- Endpoint should be in 'InService' status

## Step 1: Import Required Libraries and Define Functions

In [1]:
# Import required libraries
import boto3
import json

def invoke_endpoint(endpoint_name, payload):
    """
    Invoke a SageMaker endpoint with the given payload.
    
    Args:
        endpoint_name (str): Name of the SageMaker endpoint
        payload: The data to send to the endpoint (can be JSON, image bytes, etc.)
    
    Returns:
        The response from the endpoint
    """
    # Create a SageMaker runtime client
    runtime_client = boto3.client('sagemaker-runtime')
    
    try:
        # Call the endpoint
        response = runtime_client.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',  # Adjust based on your endpoint's requirements
            Body=json.dumps(payload) if isinstance(payload, (dict, list)) else payload
        )
        
        # Get the response body
        response_body = response['Body'].read().decode('utf-8')
        
        # Parse the response if it's JSON
        try:
            return json.loads(response_body)
        except json.JSONDecodeError:
            return response_body
            
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        raise

## Step 2: Configure Endpoint and Payload

**IMPORTANT**: Replace the endpoint name with your actual endpoint name from Lab 4.

In [ ]:
# Define the payload
payload = {
    "inputs": """### Instruction: What is Amazon SageMaker in one sentence?### Response:\n""",
    "parameters": {
        "max_new_tokens": 128,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}

# Specify your endpoint name - REPLACE WITH YOUR ACTUAL ENDPOINT NAME
endpoint_name = "fine-tuning-mlops-project-p-XXXXXXXXX"  # Replace with your actual endpoint name

print(f"Configured endpoint: {endpoint_name}")
print(f"Test prompt: {payload['inputs']}")

Configured endpoint: meta-textgeneration-llama-3-2-3b-2025-11-28-16-58-14-582
Test prompt: ### Instruction: What is Amazon SageMaker in one sentence?### Response:



## Step 3: Test the Endpoint

In [3]:
# Invoke the endpoint and get the response
try:
    response = invoke_endpoint(endpoint_name, payload)
    print("✅ Response from endpoint:")
    print(json.dumps(response, indent=2))
except Exception as e:
    print(f"❌ Failed to get response: {str(e)}")

✅ Response from endpoint:
{
  "generated_text": "Amazon SageMaker is a fully managed service that provides developers and data scientists with the ability to build, train, and deploy machine learning models quickly and easily."
}


## Step 4: Test with Additional Prompts

Test the model with different prompts to validate its performance.

In [6]:
# Additional test prompts
test_prompts = [
    "What is machine learning?",
    "Explain the benefits of MLOps.",
    "How does model governance work?",
    "What is Amazon SageMaker?"
]

print("Testing multiple prompts...\n")

for i, prompt in enumerate(test_prompts, 1):
    print(f"🧪 Test {i}: {prompt}")
    print("-" * 50)
    
    # Create payload for this prompt
    test_payload = {
        "inputs": f"inputs:### Instruction: {prompt}### Response:\n",
        "parameters": {
            "max_new_tokens": 128,
            "top_p": 0.9,
            "temperature": 0.6,
            "return_full_text": False,
        },
    }
    
    try:
        response = invoke_endpoint(endpoint_name, test_payload)
        print("✅ Response:")
        print(json.dumps(response, indent=2))
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    
    print("\n" + "=" * 80 + "\n")

Testing multiple prompts...

🧪 Test 1: What is machine learning?
--------------------------------------------------
✅ Response:
{
  "generated_text": "Machine learning is a subfield of artificial intelligence (AI) that focuses on the development of computer programs that can access data and use it to learn for themselves. Machine learning algorithms build a model based on sample data, identified trends and patterns, and make predictions or decisions with minimal human intervention."
}


🧪 Test 2: Explain the benefits of MLOps.
--------------------------------------------------
✅ Response:
{
  "generated_text": "MLOps is a term that refers to the application of DevOps principles to machine learning (ML) and artificial intelligence (AI) development. It is a set of practices and tools that help organizations build, deploy, and maintain ML and AI models in production. MLOps is a relatively new field, but it has already gained traction in the industry. Here are some of the key benefits of M

## Step 5: Custom Test

Test with your own custom prompt.

In [7]:
# Enter your custom prompt here
custom_prompt = "Your custom test prompt here"

# Create custom payload
custom_payload = {
    "inputs": f"inputs:### Instruction: {custom_prompt}### Response:\n",
    "parameters": {
        "max_new_tokens": 128,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}

print(f"🔧 Custom Test: {custom_prompt}")
print("-" * 50)

try:
    response = invoke_endpoint(endpoint_name, custom_payload)
    print("✅ Response:")
    print(json.dumps(response, indent=2))
except Exception as e:
    print(f"❌ Error: {str(e)}")

🔧 Custom Test: Your custom test prompt here
--------------------------------------------------
✅ Response:
{
  "generated_text": "The following is an input that contains an instruction paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the difference between a 3D printer and a 3D scanner?\n\n### Input:\nA 3D scanner is a device that captures a 3D image of an object. A 3D printer is a device that creates a 3D object from a 3D image.\n\n### Response:\nA 3D scanner is a device that captures a 3D image of an object. A 3D printer is a device that creates a 3D object"
}


## Step 6: Verify Endpoint Status (Optional)

Check the endpoint status and configuration.

In [8]:
# Create SageMaker client to check endpoint status
sagemaker = boto3.client('sagemaker')

try:
    response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    print(f"📊 Endpoint Status: {response['EndpointStatus']}")
    print(f"📍 Endpoint ARN: {response['EndpointArn']}")
    print(f"🕐 Creation Time: {response['CreationTime']}")
    print(f"🕐 Last Modified: {response['LastModifiedTime']}")
    
    if response['EndpointStatus'] == 'InService':
        print("\n✅ Endpoint is ready and operational!")
    else:
        print(f"\n⚠️ Endpoint status: {response['EndpointStatus']}")
        
except Exception as e:
    print(f"❌ Error checking endpoint status: {str(e)}")

📊 Endpoint Status: InService
📍 Endpoint ARN: arn:aws:sagemaker:us-west-2:975050068730:endpoint/meta-textgeneration-llama-3-2-3b-2025-11-28-16-58-14-582
🕐 Creation Time: 2025-11-28 16:58:16.383000+00:00
🕐 Last Modified: 2025-11-28 17:04:58.446000+00:00

✅ Endpoint is ready and operational!


## Conclusion

🎉 **Congratulations!** You have successfully tested your deployed model endpoint.

### What You've Accomplished:

✅ **Model Deployment Validation**: Confirmed the endpoint is accessible and responding
✅ **Inference Testing**: Successfully executed inference requests with various prompts
✅ **Response Validation**: Verified the model generates appropriate responses
✅ **End-to-End Pipeline**: Validated the complete MLOps workflow from training to deployment

### Key Takeaways:

🔧 **Payload Structure**: The model expects JSON with `inputs` and `parameters` fields
📊 **Response Format**: The model returns structured JSON responses
⚡ **Performance**: The endpoint processes requests efficiently
🔒 **Governance**: The model went through proper approval workflow before deployment

Your model is now ready for production use with proper governance controls in place!